In [4]:
import cv2
import numpy as np
import pandas as pd
import os

# Задаем путь к Excel-файлу
excel_file = 'lab6.xlsx'

# Загрузка данных из файла Excel
df = pd.read_excel(excel_file)

# Включаем нумерацию строк для выбора варианта
N = ord('D') % 5 + 1  # Определяем номер варианта
print(f"Номер варианта: {N}")

# Выбираем строку для варианта
data_variant = df.loc[df['N'] == N]
print("Данные для варианта:")
print(data_variant)

# Функция для обработки изображений и добавления очков
def process_image_from_dataframe(data_row, output_path):

    # Загрузка классификаторов
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Параметры для рисования
    color_dict = {
        'red': (255, 0, 0),
        'blue': (0, 0, 255),
        'cian': (0, 255, 255),
        'magenta': (255, 0, 255),
        'yellow': (0, 255, 255)
    }
    color_name = data_row['glasses color']
    color = color_dict.get(color_name, (0, 0, 0))  # По умолчанию черный, если цвет не найден
    thickness = data_row['line width']  # Толщина линий

    # Полный путь к изображению
    image_path = os.path.join('Images', data_row['file name'])  # Путь к файлу в папке Images

    # Загружаем изображение
    print(f'Loading image from: {image_path}')
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Изображение не найдено: {image_path}")

    # Конвертируем изображение в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Находим лица на изображении
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        # Выделяем область лица для поиска глаз
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

        # Поиск глаз
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            # Сортируем глаза по координатам X
            eyes = sorted(eyes, key=lambda e: e[0])
            left_eye = eyes[0]
            right_eye = eyes[1]

            # Координаты центров глаз
            lx, ly = (left_eye[0] + left_eye[2] // 2, left_eye[1] + left_eye[3] // 2)
            rx, ry = (right_eye[0] + right_eye[2] // 2, right_eye[1] + right_eye[3] // 2)

            # Рисуем очки
            radius = max(left_eye[2], left_eye[3]) // 2
            cv2.circle(roi_color, (lx, ly), radius, color, thickness)

            radius = max(right_eye[2], right_eye[3]) // 2
            cv2.circle(roi_color, (rx, ry), radius, color, thickness)

            # Рисуем линию между очками
            cv2.line(roi_color, (lx + radius, ly), (rx - radius, ry), color, thickness)

    # Сохраняем обработанное изображение
    face_center = img[y:y+h, x:x+w]
    reduced_img = cv2.resize(face_center, (600, 600))  # Меняем размер изображения
    cv2.imwrite(output_path, reduced_img)
    print(f"Изображение сохранено по пути: {output_path}")

# Вызов функции для выбранного варианта
output_path = 'result.jpg'
process_image_from_dataframe(data_variant.iloc[0], output_path)


Номер варианта: 4
Данные для варианта:
   N       file name image size glasses color  line width
3  4  longbottom.jpg    600x600       magenta           5
Loading image from: Images\longbottom.jpg
Изображение сохранено по пути: result.jpg
